In [ ]:
import random

In [ ]:
!git clone https://github.com/jack0713323/GCNII.git
%cd GCNII

In [ ]:
!ls  

選擇要使用的資料庫

設定參數、代數、等等

In [ ]:
Ngen=10
Nsol=10

In [ ]:
random_number_range=[(2,64),(0,4),(0,2),(0,3),(50,300),(500,1600),(8,64),(0,2)
                    ,(0.01,0.5),(0.01,0.7),(0.01,0.9),(0.001,0.01),(0.001,0.05),(1e-5,1e-3)]


best_num_layers = int(a[0])
best_optim = int(a[1])
best_al = int(a[2])
best_act = int(a[3])
best_earlystop = int(a[4])
best_epoch1 = int(a[5])
best_hidden_dim = int(a[6])
best_variant = a[7]

best_alpha = a[8]
best_lamda = a[9]
best_dropout = a[10]
best_lr = a[11]
best_weight_decay_1 = a[12]
best_weight_decay_2 = a[13]


In [ ]:
def random_select(n):
    return random.uniform(random_number_range[n][0],random_number_range[n][1])
def generate_random_numbers(number_range):
    random_numbers = []
    for min_value, max_value in number_range:
        random_value = random.uniform(min_value, max_value)
        if random_value>2:
          random_value=int(random_value)
        random_numbers.append(random_value)

    return random_numbers
# 定义范围

In [ ]:
rX=generate_random_numbers(random_number_range)
rX

設可調之網路 optimer都要改

In [ ]:
C1=0.1#GBEST區間
C2=0.5#PBEST區間
Cg=0.35
Cp=0.6
Cw=0.7

In [ ]:
Njob=8 # 表整數型參數個數
Njob2=6 # 表小數型參數個數
repeat=5

In [ ]:
#iSSO-ssoHPO（超參數優化）老師版本

import numpy as np
import time
import copy
import random
import os

random.seed()


Best_time=0

gbest_list=[]

start_time = time.time()
'''----- 生成初始种群 -----'''
X, F, pX, pF, gBest, genBest= [], [], [], [], 0, 0
value_to_x_dict = {}
umax=np.array([0]*(Njob+Njob2+1), dtype=np.float64)
umin=np.array([1]*(Njob+Njob2+1), dtype=np.float64)
for job in range(Njob,Njob+Njob2):
    umax[job]=random_number_range[job][1]
    umin[job]=random_number_range[job][0]
#初始解
for sol in range(Nsol):
    rX = generate_random_numbers(random_number_range)
    rX[0] = int(rX[0])
    rX[1] = int(rX[1])
    rX[2] = int(rX[2])
    rX[3] = int(rX[3])
    rX[4] = int(rX[4])
    rX[5] = int(rX[5])
    rX[6] = int(rX[6])
    rX[7] = int(rX[7])

    X.append(rX) 
    pX.append(rX) 
    print("0",sol,X[sol])
    best_num_layers = (X[sol][0])
    best_optim = (X[sol][1])
    best_al = (X[sol][2])
    best_act = (X[sol][3])
    best_earlystop = (X[sol][4])
    best_epoch1 = (X[sol][5])
    best_hidden_dim = (X[sol][6])
    best_variant = (X[sol][7])

    best_alpha = X[sol][8]
    best_lamda = X[sol][9]
    best_dropout = X[sol][10]
    best_lr = X[sol][11]
    best_weight_decay_1 = X[sol][12]
    best_weight_decay_2 = X[sol][13]

    best_test_acc_list=[]
    for k in range(repeat):
        if best_al==0 and best_variant==0:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2} 
        elif best_al==1 and best_variant==0:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --alpha {best_alpha} --lamda {best_lamda} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2} 
        elif best_al==0 and best_variant==1:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2}  --variant
        else:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --alpha {best_alpha} --lamda {best_lamda} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2}  --variant
        b=float(b[-1][-4:].replace(":", ""))
        best_test_acc_list.append(b*0.01)
        if np.mean(best_test_acc_list)<0.8:
            break
    value=np.mean(best_test_acc_list)
    print(value,)
    F.append(value)
    pF.append(value)
    if F[sol]>F[gBest]:
        gBest=sol
        #pX[gBest]=rX
        #gbest_list=best_test_acc_list
    
    value_to_x_dict[value] = (tuple(X[sol]),(0,sol))

for gen in range(1,Ngen+1):
  for sol in range(Nsol):
      temp_pX=tuple(pX[sol])
      u = (umax-umin)/2*Njob2 
      for job in range(Njob):
        rnd2=np.random.rand()
        if rnd2<Cg:
          X[sol][job]=pX[gBest][job]
        elif rnd2<Cp:
          X[sol][job]=pX[sol][job]
        elif rnd2<Cw:
          X[sol][job]=X[sol][job]
        else:
          X[sol][job]=int(random_select(job))
      for job in range(Njob,Njob+Njob2):
        rnd=np.random.rand()
        if rnd<C1 or (X[sol][job]==pX[gBest][job]):
            X[sol][job]=X[sol][job]+u[job]*(np.random.rand()-0.5)                      *(random_number_range[job][0])
        elif rnd<C2 and (X[sol][job]!=pX[gBest][job]):
            X[sol][job]=pX[gBest][job]+u[job]*(np.random.rand()-0.5)                   *(random_number_range[job][0])
        else:
            X[sol][job]=X[sol][job]+(X[sol][job]-pX[gBest][job])*(np.random.rand()-0.5)*(random_number_range[job][0])
        if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
            X[sol][job]=random_select(job)
      print(gen,sol,X[sol])
      best_num_layers = (X[sol][0])
      best_optim = (X[sol][1])
      best_al = (X[sol][2])
      best_act = (X[sol][3])
      best_earlystop = (X[sol][4])
      best_epoch1 = (X[sol][5])
      best_hidden_dim = (X[sol][6])
      best_variant = (X[sol][7])

      best_alpha = X[sol][8]
      best_lamda = X[sol][9]
      best_dropout = X[sol][10]
      best_lr = X[sol][11]
      best_weight_decay_1 = X[sol][12]
      best_weight_decay_2 = X[sol][13]
      best_test_acc_list=[]
      for k in range(repeat):
        if best_al==0 and best_variant==0:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2} 
        elif best_al==1 and best_variant==0:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --alpha {best_alpha} --lamda {best_lamda} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2} 
        elif best_al==0 and best_variant==1:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2} --variant
        else:
            b=!python -u train3.py --data cora --layer {best_num_layers} --op {best_optim}  --act_fn {best_act} --patience {best_earlystop} --epochs  {best_epoch1} --hidden {best_hidden_dim} --alpha {best_alpha} --lamda {best_lamda} --dropout {best_dropout} --lr {best_lr} --wd1 {best_weight_decay_1} --wd2 {best_weight_decay_2} --variant
        b=float(b[-1][-4:].replace(":", ""))
        best_test_acc_list.append(b*0.01)
        if np.mean(best_test_acc_list)+0.0043<pF[gBest]:
            break
      value=np.mean(best_test_acc_list)
      print(value,"vs",pF[gBest])
      pX[sol]=list(temp_pX)    
      F[sol]=value
      #計算value結束
      value_to_x_dict[value] = (tuple(X[sol]),(gen,sol))
      
      if F[sol]>pF[sol]:
        for job in range(Njob+Njob2): 
            pX[sol][job]=X[sol][job] 
        pF[sol]=F[sol]
        if F[sol]>pF[gBest]:
          gBest=sol
          genBest=gen
end_time = time.time()
sso_time1=end_time-start_time
sol0=value_to_x_dict[pF[gBest]][0]
print("optimal sequence",value_to_x_dict[pF[gBest]][0])
print("optimal value:%f"%pF[gBest])

print("gbest_time:%f"%Best_time)
print("optimal generation number:", value_to_x_dict[pF[gBest]][1][0])
print("optimal sol number:", value_to_x_dict[pF[gBest]][1][1])
print("sso_time:", sso_time1)
